# Data Preparation

In [4]:
DATA_TRAIN_PATH = "../../data/nlp/summ_train.json"
DATA_TEST_PATH = "../../data/nlp/summ_test.json"

In [5]:
import pandas as pd

train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:40000]
len(train_df)

40000

In [6]:
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:5000]
len(test_df)

5000

텍스트 요약을 위해 BART 에 들어갈 형태로 전처리 수행
* Encoder : 원문
* Decoder :  요약문

In [7]:
def preprocess_data(data):
    outs = []
    for doc in data["documents"]:
        line = []
        line.append(doc["media_name"])
        line.append(doc["id"])
        para = []
        for sent in doc["text"]:
            for s in sent:
                para.append(s["sentence"])
        line.append(para)
        line.append(doc["abstractive"][0])
        line.append(doc["extractive"])
        a = doc["extractive"]
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ["media", "id", "article_original", "abstractive", "extractive"]
    return outs_df

In [9]:
# 원문과 요약문을 각각 article_original, abstractive 에 저장한 상태
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [10]:
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


In [12]:
# 내용이 string 으로 되어있는지 확인 -> 확인 결과 List 였다
train_data["article_original"].loc[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [13]:
# 각 문장을 원소로 가지는 list 를 한 줄의 문장으로 합쳐주기
join_text = lambda x: " ".join(x)

train_data["news"] = [join_text(text) for text in train_data["article_original"]]
test_data["news"] = [join_text(text) for text in test_data["article_original"]]

In [15]:
train_data["news"].loc[0]

'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

In [16]:
train_data["abstractive"].loc[0]

"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 ."

# Dataset 생성
* tokenization 이 가능하도록 클래스화
* 각종 전처리 수행

In [17]:
import numpy as np
from torch.utils.data import DataLoader, Dataset


class KoBARTSummaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        # self.docs = pd.read_csv(file, sep='\t')
        self.docs = df
        self.len = self.docs.shape[0]

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        # 인코더의 입력과 디코더의 입력은 0으로 패딩
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[: self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        # 디코더의 레이블은 -100으로 패딩
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[: self.max_len]

        return inputs

    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]

        # ==========원문==========
        # 뉴스 기사를 정수 인코딩
        input_ids = self.tokenizer.encode(instance["news"])
        # 패딩을 적용하여 시퀀스 길이를 max_len으로 맞춤
        input_ids = self.add_padding_data(input_ids)

        # ==========요약==========
        # 요약문을 정수 인코딩 -> [요약할, 문장]
        label_ids = self.tokenizer.encode(instance["abstractive"])
        # 요약문 끝에 <eos> 토큰을 추가 -> [요약할, 문장, <eos>]
        label_ids.append(self.tokenizer.eos_token_id)

        # ==========Decoder에 들어갈 데이터==========
        # 디코더 입력 시퀀스의 첫 번째 토큰으로 <eos> 추가
        dec_input_ids = [self.tokenizer.eos_token_id]
        # 교사 강요(Teacher Forcing)을 위해 label_ids에서 마지막 토큰(<eos>)을 제외한 부분을 추가 -> [<eos>, 요약할, 문장]
        dec_input_ids += label_ids[:-1]
        # 패딩을 적용하여 시퀀스 길이를 max_len으로 맞춤
        dec_input_ids = self.add_padding_data(dec_input_ids)
        # 레이블 시퀀스에도 패딩을 적용, 패딩된 부분은 ignore_index로 설정
        label_ids = self.add_ignored_data(label_ids)

        return {
            "input_ids": np.array(input_ids, dtype=np.int_),
            "decoder_input_ids": np.array(dec_input_ids, dtype=np.int_),
            "labels": np.array(label_ids, dtype=np.int_),
        }

    def __len__(self):
        return self.len

# 모델 정의 

In [19]:
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast


class KoBARTConditionalGeneration(torch.nn.Module):
    def __init__(self):
        super(KoBARTConditionalGeneration, self).__init__()
        # KoBART 모델을 불러오고 초기화합니다.
        self.model = BartForConditionalGeneration.from_pretrained(
            "gogamza/kobart-base-v1"
        )
        # KoBART에 맞는 토크나이저를 불러옵니다.
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(
            "gogamza/kobart-base-v1"
        )
        # 패딩 토큰 ID를 설정합니다.
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):
        # 예시:
        # inputs['input_ids'] = [100, 101, 102, 103, 0, 0, 0] (원문 시퀀스)
        # inputs['decoder_input_ids'] = [2, 200, 201, 202, 0, 0, 0] (<eos>로 시작하는 디코더 입력 시퀀스)
        # inputs['labels'] = [200, 201, 202, 3, -100, -100, -100] (디코더 출력 레이블)

        # 입력 시퀀스에서 패딩 토큰이 아닌 부분을 마스크로 표시 (패딩은 0, 나머지는 1)
        # 예시: attention_mask = [1, 1, 1, 1, 0, 0, 0]
        attention_mask = inputs["input_ids"].ne(self.pad_token_id).float()

        # 디코더 입력 시퀀스에서 패딩 토큰이 아닌 부분을 마스크로 표시
        # 예시: decoder_attention_mask = [1, 1, 1, 1, 0, 0, 0]
        decoder_attention_mask = (
            inputs["decoder_input_ids"].ne(self.pad_token_id).float()
        )

        # KoBART 모델을 사용해 순전파 수행
        return self.model(
            input_ids=inputs["input_ids"],
            attention_mask=attention_mask,
            decoder_input_ids=inputs["decoder_input_ids"],
            decoder_attention_mask=decoder_attention_mask,
            labels=inputs["labels"],
            return_dict=True,
        )


# GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU를 사용합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# KoBARTConditionalGeneration 모델을 지정된 장치에 할당합니다.
model = KoBARTConditionalGeneration().to(device)

# 예시:
# input_ids = [100, 101, 102, 103, 0, 0, 0]  # 원문 시퀀스
# decoder_input_ids = [2, 200, 201, 202, 0, 0, 0]  # <eos>로 시작하는 디코더 입력 시퀀스
# labels = [200, 201, 202, 3, -100, -100, -100]  # 실제 요약문 레이블
# 패딩 부분은 0으로 마스크되고, 모델은 이 부분을 무시하고 나머지 유효한 부분만 학습에 사용

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Hyperparameter 설정

In [20]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# 하이퍼파라미터 설정
batch_size = 32  # 한 번에 처리할 배치 크기
max_len = 512  # 입력 시퀀스의 최대 길이
num_workers = 4  # DataLoader에서 데이터를 병렬로 로드할 때 사용할 워커 수
lr = 3e-5  # 학습률(learning rate)
max_epochs = 10  # 최대 에폭 수 (데이터셋을 몇 번 반복할지)
warmup_ratio = 0.1  # 학습 초기에 warm-up을 적용할 비율

# KoBART 토크나이저를 불러옵니다.
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v1")

# 학습 데이터셋과 테스트 데이터셋을 KoBARTSummaryDataset 클래스로 생성합니다.
train_dataset = KoBARTSummaryDataset(train_data, tokenizer, max_len=max_len)
test_dataset = KoBARTSummaryDataset(test_data, tokenizer, max_len=max_len)

# DataLoader를 사용해 학습 데이터와 테스트 데이터를 배치 단위로 로드합니다.
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

# AdamW 옵티마이저를 설정합니다. 이 옵티마이저는 모델의 파라미터를 업데이트할 때 사용됩니다.
optimizer = AdamW(model.parameters(), lr=lr)

# 학습 과정에서의 전체 스텝 수를 계산합니다. (배치 수 * 에폭 수)
total_steps = len(train_loader) * max_epochs

# Cosine Annealing Warm Restarts 스케줄러를 설정합니다.
# 이 스케줄러는 학습률을 코사인 함수 형태로 조정하며, 주기적으로 warm-up을 적용합니다.
scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=int(total_steps * warmup_ratio),  # warm-up 기간의 스텝 수
    T_mult=1,  # 주기가 늘어나는 비율 (1이면 주기가 일정)
    eta_min=0,
)  # 학습률의 최솟값

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Tokenization

In [21]:
print("첫번째 샘플의 원문 텍스트 :", train_data["news"].loc[0])

첫번째 샘플의 원문 텍스트 : ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


In [22]:
print(
    "첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 :",
    train_dataset[0]["input_ids"],
)
print("정수 인코딩 및 패딩 후의 길이 :", len(train_dataset[0]["input_ids"]))

첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 : [21582   296  9770 14666 10386 14136 16266 14476 12061 10675 10872 14165
 10001 14423 19168 13758 18482 14885   296   318   304   263   303 15195
   308   296 17761   245 26818 26102  9506 14973 18193 24873 24777 27841
 25486 14185 26379 15358 14049 18193 14871 17074 14902 15055 14634 17613
 19754 18193 14871 17074 16245 19609 10443 14291 15912 14396 11786 19609
 15410 14031 10386 12061 10675 14303 20054 14048 14355 14212 15049 14291
 14553 19609 15231 16864 16365 15828 26030 19560 16617 14130 14516 16309
 12024 14396 14355 14212 10675 19696 14491 20087 12005 14770 21598 26407
 12024 14383 16167   236 12037 16248 14195 25292 26407 15888 20066 29246
 12332 21405 16053   243 14044 20169 19304 15615 14423 14573 24884 19961
 11211 18193 19819 12333 10955 14842 15141 14497 15261 14239   296 15377
 14079 13455   240 14333 19609 17901 14355 14212 15049 19696 13590 18102
 14635 14130 14423 14576 14212 19839 15623 17287 17400 14056  9584 20466
 14537 14212 15689

In [23]:
print("첫번째 샘플의 정수 인코딩 후 복원 결과 :")
tokenizer.decode(train_dataset[0]["input_ids"])

첫번째 샘플의 정수 인코딩 후 복원 결과 :


'ha당 조사료 400만원...작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [24]:
print("첫번째 샘플의 요약문:", train_data["abstractive"].loc[0])

첫번째 샘플의 요약문: 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .


In [25]:
print(
    "첫번째 샘플의 요약문 텍스트의 정수 인코딩 및 패딩 결과",
    train_dataset[0]["decoder_input_ids"],
)
print("정수 인코딩 및 패딩 후의 길이 :", len(train_dataset[0]["decoder_input_ids"]))

첫번째 샘플의 요약문 텍스트의 정수 인코딩 및 패딩 결과 [    1 26102  9506 14973 18193 24873 24777 27841 25486 14185 26379 19609
 10443 14291 15912 14396 11786 19609 15410 14031 10386 12061 10675 14303
 20054 14048 14355 14212 15049 14291 14553 19609 15231 16864 16365 15828
 26030 19560 14063 11495 14871 17074 12147 15127 17489 15358 15272 14432
 14834 15271 15243 15869 15450 15364 14497 12332 16516 12332 23891 10586
  9879 17982 18290 15453 17210  9754 17546     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3 

In [26]:
print("첫번째 샘플의 정수 인코딩 및 패딩 후 복원 결과 :")
tokenizer.decode(train_dataset[0]["decoder_input_ids"])

첫번째 샘플의 정수 인코딩 및 패딩 후 복원 결과 :


"</s> 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [27]:
# -100의 값은 tokenizer_decode하면 에러나므로 임시로 3으로 변경 후 출력
test_array = train_dataset[0]["labels"]
test_array[test_array == -100] = 3
print("첫번째 샘플의 요약문 레이블 :", tokenizer.decode(test_array))

첫번째 샘플의 요약문 레이블 : 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

# Training

In [ ]:
from tqdm import tqdm

best_loss = np.inf
for epoch in range(max_epochs):
    print(epoch + 1, "수행 중")
    model.train()
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {
            k: v.to(device) for k, v in batch.items()
        }  # Move the batch tensors to the same device as the model
        optimizer.zero_grad()
        outputs = model(batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(test_loader, total=len(test_loader)):
            batch = {
                k: v.to(device) for k, v in batch.items()
            }  # Move the batch tensors to the same device as the model
            outputs = model(batch)
            total_loss += outputs.loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss}")

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "best_model.pt")
        print(
            f"Validation loss improved from {best_loss:.4f} to {avg_loss:.4f}. 체크포인트를 저장합니다."
        )

In [ ]:
# 모델 인스턴스 생성
model_wrapper = KoBARTConditionalGeneration().to(device)

# 가중치 로드
model_wrapper.load_state_dict(torch.load("best_model.pt"))

# 모델을 평가 모드로 설정
model_wrapper.eval()

In [ ]:
text = test_data.loc[25]["news"]
text

In [ ]:
label = test_data.loc[25]["abstractive"]
label

In [ ]:
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0).to(device)
output = model_wrapper.model.generate(
    input_ids, eos_token_id=1, max_length=512, num_beams=5
)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)